<h1 style="text-align: center;">Extreme growth in terrorist threat in Europe</h1>

<p style="text-align: center;">A story by Brechje van Lokhorst,<br>
    Jada Xie, Nils Böhne and Yochem van Rosmalen.</p>

In [19]:
import pandas as pd
import numpy as np
from math import isnan

from plotly.offline import init_notebook_mode, iplot
import plotly.graph_objs as go

# Start notebook mode
init_notebook_mode(connected=True)
pd.options.mode.chained_assignment = None

def get_data():
    raw_data = pd.read_csv('dataset.csv', encoding='ISO-8859-1', low_memory=False)

    selected_columns = raw_data[[
        'eventid',
        'iyear',
        'imonth',
        'iday',
        'country_txt',
        'region_txt',
        'gname',
        'nkill',
        'motive',
        'attacktype1_txt',
        'weaptype1_txt',
        'nwound',
        'nhostkid',
        'success',
        'natlty1_txt'
    ]]
    
    selected_columns['nwound'] = selected_columns['nwound'].fillna(0).astype(int)
    selected_columns['nkill'] = selected_columns['nkill'].fillna(0).astype(int)
    selected_columns['casualities'] = selected_columns['nkill'] + selected_columns['nwound']
    selected_columns['amount'] = 1

    return selected_columns

Terrorism seems to be on the rise. The devastating ISIS attack 
on Paris in 2015 killed 130, the truck attack in 
Nice just one year after had 84 fatalities. More people in 
Europe are on alert about public gatherings and are 
uneasy about islamist groups. Has the number of terrorism cases 
truely risen in Europe? Or is it just the recency 
effect exaggerating the situation?

Global Terrorism Database is the one we are working with to 
tackle this issue. It keeps track of incidents of 
terrorism from 1970 onward, and it’s maintained by the National 
Consortium for the Study of Terrorism and 
Responses to Terrorism (START). Overall, the terrorist attacks 
in Europe from 1970 to 2017 have grown in numbers, 
in the fatality and wound amounts, as well as fatality rates.

In [ ]:
def total_n_wounded_killed():
    data = get_data()
    data = data[(data.region_txt == 'Eastern Europe') | \
                (data.region_txt == 'Western Europe')]
    
    casul_per_year = {x.iyear: [0, 0] for x in data.itertuples()}
    
    for a in data.itertuples():
        casul_per_year[a.iyear][0] += a.nwound
        casul_per_year[a.iyear][1] += a.nkill

    casul_year = sorted([(year, casul[0], casul[1]) for year, casul in casul_per_year.items()])

    years = [x[0] for x in casul_year]
    wounded = [x[1] for x in casul_year]
    deaths = [x[2] for x in casul_year]

    # Create a trace
    trace1 = go.Scatter(
        x = years,
        y = deaths,
        name = 'Deaths'
    )
    trace2 = go.Scatter(
        x = years,
        y = wounded,
        name = 'Wounded'
    )

    layout = go.Layout(
        title = 'Number of casualties per year of terrorist attacks in Europe',
        xaxis=go.layout.XAxis(
            title='Year',
        ),
        yaxis = go.layout.YAxis(
            title='Count'
        ),
    )

    data = [trace1, trace2]

    fig = go.Figure(data=data, layout=layout)
    iplot(fig, filename='')

total_n_wounded_killed()

Looking closer, we find that 2014 is an exceptionally bad year, 
where the number of deaths caused by terrorist attacks rose by 
almost three folds from the last data peak in 1999. To see 
whether this spike in death rate signals a change in the 
severity level of these attacks, we calculated the total kill 
rate divided by the total wound rate per year in Europe from 
1970 to 2017, and from 2005 to 2017. 

In [ ]:
def wounded_year():
    data = get_data()
    j = data[data['region_txt'] == 'Western Europe']
    i = data[data['region_txt'] == 'Eastern Europe']
    e = pd.concat([i, j])
    e = e[e['nkill'] != 0]
    e = e[e['nwound'] != 0]
    trace = go.Scatter(
        x = e['iyear'],
        y = e['nkill'] / e['nwound'],
        mode = 'markers',
        marker=dict(
            size=8,
            color= e['nkill'] / e['nwound'],
            colorscale='Viridis',
            showscale = True
        )
    )

    layout= go.Layout(
        title= 'Fatality per wounded in Europe (1970 - 2017)',
        xaxis= dict(
            title= 'Year',
        ),
        yaxis=dict(
            title= 'Killed / wounded',
        ),
        showlegend= False
    )

    fig= go.Figure(data=[trace], layout=layout)
    iplot(fig)

wounded_year()

The graphs clearly show that there has been an increase in attacks
that are high in severity over the  long term. Over the last decade 
however, the increase is more subtle. 

In [ ]:
def wounded_short_year():
    data = get_data()
    j = data[data['region_txt'] == 'Western Europe']
    i = data[data['region_txt'] == 'Eastern Europe']
    e = pd.concat([i, j])
    e = e[e['nkill'] != 0]
    e = e[e['nwound'] != 0]
    e = e[e['iyear'] > 2004]
    trace = go.Scatter(
        x = e['iyear'],
        y = e['nkill'] / e['nwound'],
        mode = 'markers',
        marker=dict(
            size=8,
            color=e['nkill'] / e['nwound'],                # set color to an array/list of desired values
            colorscale='Viridis',   # choose a colorscale
            opacity=0.8,
            showscale = True
        )
    )

    layout= go.Layout(
        title= 'Fatality per wounded in Europe (2005 - 2017)',
        xaxis= dict(
            title= 'Year',
        ),
        yaxis=dict(
            title= 'Killed / wounded',
        ),
        showlegend= False
    )

    fig= go.Figure(data=[trace], layout=layout)
    iplot(fig)

wounded_short_year()

There are 9 types of attack types in the database: 
assasination, hostage taking (kidnapping), bombing/explosion, 
facility/infrastructure attack, armed assault, hijacking, 
unknown, unarmed assault, hostage taking (barricade incident). 
We see a huge spike of both bombing/explosion and armed assault 
in 2014, and the cases of bombing/explosions remained high in 
2015. 

In [ ]:
def frequency_attacktype():
    data = get_data()
    i = data[data['region_txt'] == 'Eastern Europe']
    j = data[data['region_txt'] == 'Western Europe']
    p = pd.concat([i, j])
    p = p[p['iyear'] > 2004]
    p = p[p['nwound'] != 0]
    p = p[p['nkill'] != 0]
    a = p[p['attacktype1_txt'] == 'Assassination']
    b = p[p['attacktype1_txt'] == 'Hostage Taking (Kidnapping)']
    c = p[p['attacktype1_txt'] == 'Bombing/Explosion']
    d = p[p['attacktype1_txt'] == 'Facility/Infrastructure Attack']
    e = p[p['attacktype1_txt'] == 'Armed Assault']
    f = p[p['attacktype1_txt'] == 'Hijacking']
    g = p[p['attacktype1_txt'] == 'Unknown']
    h = p[p['attacktype1_txt'] == 'Unarmed Assault']
    i = p[p['attacktype1_txt'] == 'Hostage Taking (Barricade Incident)']
    
    trace0 = go.Histogram(x=a['iyear'], nbinsx=13, name='Explosives', marker=dict(color='#ffdfd3'), histnorm='', opacity=0.75)
    trace1 = go.Histogram(x=b['iyear'], nbinsx=13, name='Firearms', marker=dict(color='#957dad'), histnorm='', opacity=0.75)
    trace2 = go.Histogram(x=c['iyear'], nbinsx=13, name='Melee', marker=dict(color='#d291bc'), histnorm='', opacity=0.75)
    trace3 = go.Histogram(x=d['iyear'], nbinsx=13, name='Unknown', marker=dict(color='#fec8d8'), histnorm='', opacity=0.75)
    trace4 = go.Histogram(x=e['iyear'], nbinsx=13, name='Incendiary', marker=dict(color='#e0bbe4'), histnorm='', opacity=0.75)
    trace5 = go.Histogram(x=f['iyear'], nbinsx=13, name='Other', marker=dict(color='orange'), histnorm='', opacity=0.75)
    trace6 = go.Histogram(x=g['iyear'], nbinsx=13, name='Vehicle', marker=dict(color='purple'), histnorm='', opacity=0.75)

    layout = go.Layout(
        title="Frequency count for weapon type over the years in Europe",
        barmode='stack',
        xaxis= dict(
            title= 'Year',
        ),
        yaxis=dict(
            title= 'Frequency',
        ),
        )


    fig = go.Figure(data=[trace0, trace1, trace2, trace3, trace4, trace5, trace6], layout=layout)

    iplot(fig)

frequency_attacktype()

Comparing the percentage of weapon types in 2005, 2010 and 2014 
(3 of the 4 peak years from the frequency graph) we see that 
the percentage of explosives used slightly decreased (68.7% → 
64.9% →  55.9%) and firearm usage slightly increased (15.6% → 
24.4% → 21.8%). The type *Unknown*, however, experienced a 
notable increase  (0% → 0.509% → 10.3%). The weapon types used 
over the years haven’t changed much, the increased fatality are 
mostly due to the frequency of attacks. 

In [ ]:
def weapontype():
    data = get_data()
    
    i = data[data['region_txt'] == 'Eastern Europe']
    j = data[data['region_txt'] == 'Western Europe']
    p = pd.concat([i, j])

    q = p[p['iyear'] == 2005]
    r = p[p['iyear'] == 2014]
    x = p[p['iyear'] == 2010]

    #2005
    h = q[q['weaptype1_txt'] == 'Explosives']
    it = q[q['weaptype1_txt'] == 'Firearms']
    jt = q[q['weaptype1_txt'] == 'Melee']
    k = q[q['weaptype1_txt'] == 'Unknown']
    l = q[q['weaptype1_txt'] == 'Incendiary']
    m = q[q['weaptype1_txt'] == 'Other']
    n = q[q['weaptype1_txt'] == 'Vehicle (not to include vehicle-borne explosives, i.e., car or truck bombs)']
    
    #2010
    o = x[x['weaptype1_txt'] == 'Explosives']
    pt = x[x['weaptype1_txt'] == 'Firearms']
    s = x[x['weaptype1_txt'] == 'Melee']
    t = x[x['weaptype1_txt'] == 'Unknown']
    u = x[x['weaptype1_txt'] == 'Incendiary']
    v = x[x['weaptype1_txt'] == 'Other']
    w = x[x['weaptype1_txt'] == 'Vehicle (not to include vehicle-borne explosives, i.e., car or truck bombs)']

    #2014
    a = r[r['weaptype1_txt'] == 'Explosives']
    b = r[r['weaptype1_txt'] == 'Firearms']
    c = r[r['weaptype1_txt'] == 'Melee']
    d = r[r['weaptype1_txt'] == 'Unknown']
    e = r[r['weaptype1_txt'] == 'Incendiary']
    f = r[r['weaptype1_txt'] == 'Other']
    g = r[r['weaptype1_txt'] == 'Vehicle (not to include vehicle-borne explosives, i.e., car or truck bombs)']
    
    trace0 = go.Pie(
        labels = ['Explosives','Firearms','Melee','Unknown', 'Incendiary', 'Other', 'Vehicle'],
        values = [h.shape[0],it.shape[0],jt.shape[0],k.shape[0], l.shape[0], m.shape[0],n.shape[0]],
        hoverinfo = 'label+percent',
        visible = True,
        sort = False
        )

    trace1 = go.Pie(
        labels = ['Explosives','Firearms','Melee','Unknown', 'Incendiary', 'Other', 'Vehicle'],
        values = [o.shape[0],pt.shape[0],s.shape[0],t.shape[0], u.shape[0], v.shape[0], w.shape[0]],
        hoverinfo = 'label+percent',
        visible = False,
        sort = False
        )
    
    trace2 = go.Pie(
            labels = ['Explosives','Firearms','Melee','Unknown', 'Incendiary', 'Other', 'Vehicle'],
        values = [a.shape[0],b.shape[0],c.shape[0],d.shape[0], e.shape[0], f.shape[0],g.shape[0]],
        hoverinfo = 'label+percent',
        visible = False,
        sort = False
        )
    
    data = [trace0, trace1, trace2]
    s = [2005, 2010, 2014]

    steps = []
    for i in range(len(data)):
        step = dict(
            method = 'restyle',  
            args = ['visible', [False] * len(data)],
            label = s[i]
        )
        step['args'][1][i] = True 
        steps.append(step)
    
    sliders = [dict(
        active = 0,
        currentvalue = {},
        pad = {},
        steps = steps
    )]

    layout = go.Layout(
        dict(sliders=sliders),
        title= "Change over the years of weapon type use",
    )

    fig = go.Figure(data=data, layout=layout)

    iplot(fig)
    
weapontype()

Looking closer at the data from 2014 and 2015, incidents in 
Ukraine are the main cause of both years, taking up 76.2% in 
2014 and 63.6% in 2015. The runner-up of the two years is the 
United Kingdom 8.77% and 11.4%, respectively. The notable 
increases are Germany and France.

In [ ]:
def attacks_where():
    data = get_data()
    
    i = data[data['region_txt'] == 'Eastern Europe']
    j = data[data['region_txt'] == 'Western Europe']
    p = pd.concat([i, j])
    q = p[p['iyear'] == 2014]
    p = p[p['iyear'] == 2015]
    
    #2014
    ax= q[q['country_txt'] == 'Bosnia-Herzegovina']
    bx= q[q['country_txt'] == 'Ukraine']
    cx= q[q['country_txt'] == 'Bulgaria']
    dx= q[q['country_txt'] == 'Albania']
    ex= q[q['country_txt'] == 'Russia']
    fx= q[q['country_txt'] == 'Macedonia']
    gx= q[q['country_txt'] == 'Kosovo']
    hx= q[q['country_txt'] == 'Montenegro']
    ix= q[q['country_txt'] == 'Hungary']
    jx= q[q['country_txt'] == 'Czech Republic']
    kx= q[q['country_txt'] == 'Estonia']
    lx= q[q['country_txt'] == 'Sweden']
    mx= q[q['country_txt'] == 'United Kingdom']
    nx= q[q['country_txt'] == 'France']
    ox= q[q['country_txt'] == 'Netherlands']
    px= q[q['country_txt'] == 'Germany']
    qx= q[q['country_txt'] == 'Greece']
    rx= q[q['country_txt'] == 'Ireland']
    sx= q[q['country_txt'] == 'Cyprus']
    tx= q[q['country_txt'] == 'Denmark']
    ux= q[q['country_txt'] == 'Malta']
    vx= q[q['country_txt'] == 'Italy']
    wx= q[q['country_txt'] == 'Finland']
    xx= q[q['country_txt'] == 'Spain']

    #2015
    a= p[p['country_txt'] == 'Russia']
    b= p[p['country_txt'] == 'Ukraine']
    c= p[p['country_txt'] == 'Hungary']
    d= p[p['country_txt'] == 'Czech Republic']
    e= p[p['country_txt'] == 'Kosovo']
    f= p[p['country_txt'] == 'Macedonia']
    g= p[p['country_txt'] == 'Albania']
    h= p[p['country_txt'] == 'Bulgaria']
    it= p[p['country_txt'] == 'Bosnia-Herzegovina']
    jt= p[p['country_txt'] == 'Greece']
    k= p[p['country_txt'] == 'Germany']
    l= p[p['country_txt'] == 'Italy']
    m= p[p['country_txt'] == 'Cyprus']
    n= p[p['country_txt'] == 'Ireland']
    o= p[p['country_txt'] == 'France']
    pt= p[p['country_txt'] == 'United Kingdom']
    qt= p[p['country_txt'] == 'Sweden']
    r= p[p['country_txt'] == 'Iceland']
    s= p[p['country_txt'] == 'Belgium']
    t= p[p['country_txt'] == 'Netherlands']
    u= p[p['country_txt'] == 'Spain']
    v= p[p['country_txt'] == 'Malta']
    
    fig = {
    'data': [
           {
            'labels': ['Bosnia-Herzegovina', 'Ukraine', 'Bulgaria', 'Albania', 'Russia','Macedonia', 'Kosovo', 'Montenegro', 'Hungary', 'Czech Republic','Estonia', 'Sweden', 'United Kingdom', 'France', 'Netherlands','Germany', 'Greece', 'Ireland', 'Cyprus', 'Denmark', 'Malta','Italy', 'Finland', 'Spain'],
            'values': [ax.shape[0],bx.shape[0],cx.shape[0],dx.shape[0], ex.shape[0], fx.shape[0],gx.shape[0], hx.shape[0], ix.shape[0], jx.shape[0], kx.shape[0], lx.shape[0], mx.shape[0], nx.shape[0], ox.shape[0], px.shape[0], qx.shape[0], rx.shape[0], sx.shape[0], tx.shape[0], ux.shape[0], vx.shape[0], wx.shape[0], xx.shape[0]],
            'type': 'pie',
            "domain": {"column": 0},
            "hoverinfo":"percent+label",
            'textinfo':'none'
           },
           {
            'labels': ['Russia', 'Ukraine', 'Hungary', 'Czech Republic', 'Kosovo','Macedonia', 'Albania', 'Bulgaria', 'Bosnia-Herzegovina', 'Greece','Germany', 'Italy', 'Cyprus', 'Ireland', 'France','United Kingdom', 'Sweden', 'Iceland', 'Belgium', 'Netherlands','Spain', 'Malta'],
            'values': [a.shape[0],b.shape[0],c.shape[0],d.shape[0],e.shape[0],f.shape[0],g.shape[0], h.shape[0], it.shape[0], jt.shape[0], k.shape[0], l.shape[0], m.shape[0], n.shape[0], o.shape[0], pt.shape[0], qt.shape[0], r.shape[0], s.shape[0], t.shape[0], u.shape[0], v.shape[0]],
            'type': 'pie',
            "domain": {"column": 1},
            "hoverinfo":"percent+label",
            'textinfo':'none'
           },
            ],
     'layout': {
        'title':'Where did the attacks take place in 2014 and 2015?',
        "grid": {"rows": 1, "columns": 2},
        "annotations": [
            {
                "font": {
                    "size": 20
                    },
                "showarrow": False,
                "text": "2014",
                "x": 0.20,
                "y": 1.1
                },
                {
                "font": {
                    "size": 20
                    },
                "showarrow": False,
                "text": "2015",
                "x": 0.8,
                "y": 1.1
                }
                        ]
        }
    }

    iplot(fig)
    
attacks_where()

The Ukrainan Revolution happened in 2014, which eventually 
ousted the elected president and the Ukrainian government. The 
large number of related attacks contributes to the peak for 
Europe in 2014 and 2015. Britain has been heavily targeted by 
jihadist groups in the last few years, with a notable instance 
of a bombing at an Ariana Grande concert (Dodd, Pidd, 
Rawlinson, Siddique, & MacAskill, 2017). The British Prime 
Minister said in 2014, “what we are facing in Iraq with Isil 
[Islamic State] is a greater threat to our security than we 
have seen before.” (Coates & Hamilton, 2014)

The threat of terrorism in Europe has risen considerably in the 
last decade with the characteristics of them being more 
frequent and more deadly. It has not been a case of fear 
mongering, and Europe is facing a real challenge.

#### References
Coates, S., & Hamilton, F. (2014, August 30). *Security 
crackdown amid severe terror threat*. Retrieved June 21, 2019, 
from https://www.thetimes.co.uk/article/security-crackdown-amid-severe-terror-threat-v0tvj2vjd38 

Dodd, V., Pidd, H., Rawlinson, K., Siddique, H., & MacAskill, 
E. (2017, May 23). *At least 22 killed, 59 injured in suicide 
attack at Manchester Arena*. Retrieved June 21, 2019, from 
https://www.theguardian.com/uk-news/2017/may/22/manchester-arena-police-explosion-ariana-grande-concert-england

*[Download](https://yochem.github.io/terrorism/story1.ipynb) 
(.ipynb) or [explore](https://github.com/yochem/terrorism/blob/master/story1.ipynb) 
the code for this data story.*